<a href="https://colab.research.google.com/github/Lenguist/insight-game-ai/blob/main/simple_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Initialization
import random
class Buyer:
    def __init__(self, maxprice):
        self.maxprice = maxprice
        self.prob_to_walk = 0
    def print_params(self):
        print(f"Buyer prob to walk {self.prob_to_walk}")

class Seller:
    def __init__(self, minprice):
        self.minprice = minprice

def check_offer(offer, buyer, seller):
  if offer >= seller.minprice and offer <= buyer.maxprice:
    return "accepted"
  else:
    random_number = random.uniform(0, 1)
    if random_number < buyer.prob_to_walk:
      return "walked"

In [ ]:
#@title Simplest Bargaining (-1 offer every time)

buyer = Buyer(15)
seller = Seller(10)
offer = 20 #initial buyer offer
print("initial offer is " + str(offer))

while offer>seller.minprice:
  print("new round")
  buyer.print_params()
  if check_offer(offer, buyer, seller)=="accepted":
    print("Deal made at " + str(offer))
    break

  if check_offer(offer, buyer, seller) == "walked":
    print(f"The buyer walked at the offer of {offer}")
    break

  else:
    print(f"Offer at {offer} rejected by the buyer")
    offer += -1    
    # faster bargaining 
    # offer = min(int((offer+buyer.maxprice)/2),offer-1)

    buyer.prob_to_walk += 0.05
    buyer.prob_to_walk = round(buyer.prob_to_walk,2)
    check_offer(offer, buyer, seller)

In [ ]:
#@title Input Bargaining (user input game demo)
def input_bargaining(seller_minprice, init_offer, product_value):
  #we define buyer_maxprice to be a random number between the product value and the initial offer
  max_price = int(random.uniform(product_value, init_offer))
  buyer = Buyer(max_price) #initialize buyer with the max price they are willing to buy at
  seller = Seller(seller_minprice) # initialize seller
  offer = init_offer
  print(f"Seller (you) make initial offer of {offer}; your break even cost is {seller_minprice}.")

  while offer>=seller.minprice:
    if check_offer(offer, buyer, seller)=="accepted":
      print("Deal made at " + str(offer))
      print("buyer max price: "+str(buyer.maxprice))
      print("missed value: "+str(buyer.maxprice-offer))
      break

    if check_offer(offer, buyer, seller) == "walked": 
      print(f"The buyer walked at the offer of {offer}")
      print("buyer max price: "+str(buyer.maxprice))
      print(f"value of car: {product_value}")
      print("unrealized profit from not selling: "+str(buyer.maxprice-product_value))
      break

    else:
      print(f"Offer at {offer} rejected by the buyer")
      offer = int(input("what is your new offer?"))
      buyer.prob_to_walk += 0.05
      buyer.prob_to_walk = round(buyer.prob_to_walk,2)
      check_offer(offer, buyer, seller)

input_bargaining(10,30,10)

In [ ]:
#@title Bargaining Strategies function, outputs (money made, value lost)
def bargaining_event(product_value, max_possible_buyer_price, printing=False):
  #printing variable is for debugging, set it to True to debug
  #we define buyer_maxprice to be a random number between the product value and max_possible_buyer_price
  max_price = int(random.uniform(product_value, max_possible_buyer_price))+1
  buyer = Buyer(max_price) #initialize buyer with the max price they are willing to buy at
  buyer_init_impatience = 0
  buyer_impatience_increment = .1
  seller = Seller(product_value+1) # initialize seller
  initial_offer = 20
  offer = initial_offer # initial offer
  if printing: 
    print("initial offer is " + str(offer))

  while offer>=seller.minprice:
    if check_offer(offer, buyer, seller)=="accepted":
      if printing:
        print(f"deal accepted at {offer}. Money made: {offer-product_value}") 
      return (offer-product_value, 0)
    if check_offer(offer, buyer, seller) == "walked": 
      if printing:
        print(f"buyer walked at {offer}. Value lost: {buyer.maxprice-product_value}") 
      return (0,buyer.maxprice-product_value)
    else: # now offer a new price
      if printing:
        print(f"offer rejected at {offer}.")
      change_in_offer = 1
      offer -= change_in_offer    #change the offer for the next time
      buyer.prob_to_walk += buyer_impatience_increment
      if printing:
        print(f"New offer is {offer}") 

print(bargaining_event(10,20,True))


initial offer is 20
offer rejected at 20.
New offer is 19
offer rejected at 19.
New offer is 18
offer rejected at 18.
New offer is 17
offer rejected at 17.
New offer is 16
buyer walked at 16. Value lost: 5
(0, 5)


In [ ]:
#@title Runs simulation $n$ times and outputs profit
value_gained = 0
n = 100000
for i in range(n):
  value_gained += bargaining_event(10,15,False)[0]

print(f"you gained ${value_gained} over {n} runs")

you gained $279213 over 100000 runs


In [ ]:
# COMPARE FASTER BARGAINING VS NORMAL FOR SEVERAL ITERATIONS